In [1]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
#from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from gensim import corpora, models
#import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
#import plotly.plotly as py
import twitter
import json
import pandas as pd

In [2]:
import tweepy
import csv
import json

In [39]:
## Attributes were collected with Twitter API and saved as npy files using this code:
def saveList(attr_list,filename):
    # the filename should have extension '.npy'
    np.save(filename,attr_list)
    print("FILE Saved!")


In [38]:
def loadList(filename):
    # the filename: the extension 'npy'
    NumPyArray=np.load(filename)
    return NumPyArray.tolist()

## combine files df with tweets, user names & /attributes/ 

In [ ]:
## Add attribute : decrption (collected and saved as npy file)

In [33]:
## load csv file with correct MBTI types
df_tweets = pd.read_csv('personalities_all_tw_1.csv', index_col = 0)

In [34]:
df_tweets.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFP,NaN,BillTooke,"[""Yes. The College as the voting block for the...",NaN,"New York, USA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/8070634314...
1,ENFP,NaN,dougie0216,"['All Hail De Gendt! #TDF2019', 'He has a powe...",NaN,"Country Aire - Elgin, Oklahoma",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1126179691...
2,ENFP,NaN,kimbetech,['Get on LIVE now. 10:30pm EST https:// share....,NaN,"Tampa, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1101342873...
3,ENFP,NaN,honeyBklein,"['(Non-white, non-conservative female.)', 'Tru...",NaN,"The Real World, USA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1130313304...
4,ENFP,NaN,tanishatray4,['Are you left handed or right handed? — Left ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/9792931110...


In [36]:
## dimensions:
print('number of rows/col in final df',df_tweets.shape)

print('number of rows with empty tweets',len(df_tweets[df_tweets['tweets']=='[]']))

print('rows in dataset',len(df_tweets[df_tweets['tweets']!='[]']))


number of rows/col in final df (13526, 17)
number of rows with empty tweets 3117
rows in dataset 10409


## load descr .npy file

In [40]:
loaded_descr = loadList('attr_npy_files/descr_file.npy')

loaded_descr[:2]

["Proud dad to Patrick, and Izzy, (RIP). Debbie's husband. Proud son of Bill & Adrienne. Devout Catholic and proud of it. #GoBills",
 'Immigrants are not animals. Mid-Continental Liberal Elitist. Happily married, employed & home owner. Opinions expressed are my own. #ENFP #GoStars']

In [17]:
len(loaded_descr)

13526

In [41]:
df_tweets['descr'] = loaded_descr

In [42]:
df_tweets.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFP,NaN,BillTooke,"[""Yes. The College as the voting block for the...",NaN,"New York, USA","Proud dad to Patrick, and Izzy, (RIP). Debbie'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/8070634314...
1,ENFP,NaN,dougie0216,"['All Hail De Gendt! #TDF2019', 'He has a powe...",NaN,"Country Aire - Elgin, Oklahoma",Immigrants are not animals. Mid-Continental Li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1126179691...
2,ENFP,NaN,kimbetech,['Get on LIVE now. 10:30pm EST https:// share....,NaN,"Tampa, FL","currently binging: liberty, #climatedebate, ec...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1101342873...
3,ENFP,NaN,honeyBklein,"['(Non-white, non-conservative female.)', 'Tru...",NaN,"The Real World, USA",AdoreHubby🐊Mom of 8🐝g'mom🐣artist🎨writer🖋prolif...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1130313304...
4,ENFP,NaN,tanishatray4,['Are you left handed or right handed? — Left ...,NaN,NaN,Them eyebrows on feature. YouTuber in process ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/9792931110...


In [43]:
df_tweets.shape

(13526, 17)

## Add followers:

In [44]:
loaded_fol = loadList('attr_npy_files/followers_file.npy')

len(loaded_fol)

13526

#### The numbers in followers are text, not integers.

In [22]:
loaded_fol_int = [int(a) if a != 'NA' else -1.0 for a in loaded_fol]

In [23]:
sum(pd.isna(loaded_fol_int))

0

In [24]:
sum([1.0 if k =='NA' else 0 for k in loaded_fol_int ])

0

In [25]:
len(loaded_fol_int)

13526

In [26]:
type(loaded_fol_int[1])

int

In [45]:
df_tweets['followers'] = loaded_fol_int

In [46]:
df_tweets.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFP,NaN,BillTooke,"[""Yes. The College as the voting block for the...",NaN,"New York, USA","Proud dad to Patrick, and Izzy, (RIP). Debbie'...",NaN,712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/8070634314...
1,ENFP,NaN,dougie0216,"['All Hail De Gendt! #TDF2019', 'He has a powe...",NaN,"Country Aire - Elgin, Oklahoma",Immigrants are not animals. Mid-Continental Li...,NaN,412.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1126179691...
2,ENFP,NaN,kimbetech,['Get on LIVE now. 10:30pm EST https:// share....,NaN,"Tampa, FL","currently binging: liberty, #climatedebate, ec...",NaN,601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1101342873...
3,ENFP,NaN,honeyBklein,"['(Non-white, non-conservative female.)', 'Tru...",NaN,"The Real World, USA",AdoreHubby🐊Mom of 8🐝g'mom🐣artist🎨writer🖋prolif...,NaN,4947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1130313304...
4,ENFP,NaN,tanishatray4,['Are you left handed or right handed? — Left ...,NaN,NaN,Them eyebrows on feature. YouTuber in process ...,NaN,317.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/9792931110...


### So we have the df with several attributes to work with but ... 

### That is what happen when I saved it as csv and reopen

In [48]:
df_tw_loc_desc_fol_im = df_tweets.to_csv('personalities_all_tw_2.csv',header=True,index = True)

In [49]:
ddff = pd.read_csv('personalities_all_tw_2.csv', index_col = 0)

In [50]:
type(ddff)

pandas.core.frame.DataFrame

In [51]:
### it is different number of rows!
ddff.shape

(13532, 17)

In [120]:
ddff.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFP,NaN,BillTooke,"[""Yes. The College as the voting block for the...",NaN,"New York, USA","Proud dad to Patrick, and Izzy, (RIP). Debbie'...",NaN,712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/8070634314...
1,ENFP,NaN,dougie0216,"['All Hail De Gendt! #TDF2019', 'He has a powe...",NaN,"Country Aire - Elgin, Oklahoma",Immigrants are not animals. Mid-Continental Li...,NaN,412.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1126179691...
2,ENFP,NaN,kimbetech,['Get on LIVE now. 10:30pm EST https:// share....,NaN,"Tampa, FL","currently binging: liberty, #climatedebate, ec...",NaN,601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1101342873...
3,ENFP,NaN,honeyBklein,"['(Non-white, non-conservative female.)', 'Tru...",NaN,"The Real World, USA",AdoreHubby🐊Mom of 8🐝g'mom🐣artist🎨writer🖋prolif...,NaN,4947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/1130313304...
4,ENFP,NaN,tanishatray4,['Are you left handed or right handed? — Left ...,NaN,NaN,Them eyebrows on feature. YouTuber in process ...,NaN,317.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/9792931110...
